First lets do the same setup as we did in the basic "Using_SHEEP" notebook - to ensure we have the correct paths, and import the sheep_client.

In [1]:
import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","frontend")
import sys
sys.path.append(SHEEP_HOME)

from pysheep import sheep_client

DB LOCATION IS /Users/nbarlow/SHEEP/frontend/pysheep/sheep.db


Reset everything on the server to start a new test.

In [9]:
sheep_client.new_job()

{'content': '', 'status_code': 200}

Some HE libraries, such as HElib, support "slots", i.e. the inputs can be vectors rather than single values, and the calculation is performed simultaneously on all elements of the vectors (this is known as SIMD).  Let's instantiate an HElib context, and see how many slots we can use with the default set of parameters:


In [10]:
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int8_t")
sheep_client.get_nslots()

{'content': {'nslots': 4}, 'status_code': 200}

We can only use 4 slots with this set of parameters, but we can easily choose another set of parameters that might give us more:

In [11]:
sheep_client.set_parameters({"BaseParamSet": 2})
sheep_client.get_nslots()

{'content': {'nslots': 96}, 'status_code': 200}

We might not need all 96 slots - if we specify fewer, the server will fill the rest up with zeros.  ***However*** we do need to specify the same number of values for each of our inputs, otherwise we will get an error, e.g.:

In [12]:
sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-ADD-1.sheep"))
sheep_client.get_inputs()

{'content': ['input_0', 'input_1'], 'status_code': 200}

In [13]:
# see what happens when we put 4 values in input_0 and 3 in input_1
sheep_client.set_inputs({"input_0": [4,5,6,7], "input_1": [5,4,3]})

{'content': 'Inputs are wrong type, or different lengths', 'status_code': 500}

OK, lets try again, with equal length lists:

In [14]:
sheep_client.set_inputs({"input_0": [4,5,6,7], "input_1": [5,4,3,2]})

{'content': '', 'status_code': 200}

Now lets see if we can do the calculation - it should do element-wise addition of the two inputs, so the expected output should be "[9,9,9,9]"

In [15]:
sheep_client.run_job()

{'content': '', 'status_code': 200}

In [16]:
sheep_client.get_results()

{'content': {'cleartext check': {'is_correct': True},
  'outputs': {'output_0': ['9,9,9,9']},
  'timings': {'decryption': '276.883000',
   'encryption': '779.084000',
   'evaluation': '115.289000'}},
 'status_code': 200}